In [12]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [13]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197,5호선 상일동,373.362877,NaN
1,12,28480888,옥돌영양탕,NaN,Q,음식,Q05,닭/오리요리,Q05A01,삼계탕전문,...,143220,4945.0,NaN,1,NaN,127.089673,37.557977,5호선 아차산(어린이대공원후문),354.921957,NaN
2,14,17175285,은진이네,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,121861,4206.0,NaN,NaN,NaN,126.956164,37.551340,5호선 애오개,53.083100,NaN
3,15,17175333,흥부보쌈,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,158721,7997.0,NaN,5,NaN,126.875987,37.529132,5호선 오목교(목동운동장앞),353.540138,NaN
4,28,20665661,가파도참치,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,158050,8005.0,NaN,2,NaN,126.873242,37.524340,5호선 오목교(목동운동장앞),122.836352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,135800,17212093,만랩커피,오금역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138800,5702.0,NaN,NaN,103,127.125353,37.502381,5호선 오금,124.273859,NaN
20045,135805,17218896,얼음과자,NaN,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,157857,7510.0,NaN,1,123,126.812418,37.577436,5호선 방화,27.430119,NaN
20046,135806,17218871,오이소서울,상일점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134090,5276.0,NaN,1,104,127.165574,37.550924,5호선 상일동,983.009054,NaN
20047,135807,17218873,얼음왕국명일14,6호점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134780,5297.0,상가,1,106,127.146217,37.546317,5호선 굽은다리(강동구민회관앞),347.273163,NaN


In [14]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,NaN,상일동,상일동역 도전최강달인왕만두
1,옥돌영양탕,닭/오리요리,삼계탕전문,서울특별시 광진구 긴고랑로36길 24,5호선 아차산(어린이대공원후문),354.921957,NaN,아차산,아차산역 옥돌영양탕
2,은진이네,한식,한식/백반/한정식,서울특별시 마포구 마포대로 196,5호선 애오개,53.083100,NaN,애오개,애오개역 은진이네
3,흥부보쌈,한식,족발/보쌈전문,서울특별시 양천구 목동동로 309,5호선 오목교(목동운동장앞),353.540138,NaN,오목교,오목교역 흥부보쌈
4,가파도참치,일식/수산물,참치전문점,서울특별시 양천구 오목로54길 2,5호선 오목교(목동운동장앞),122.836352,NaN,오목교,오목교역 가파도참치
...,...,...,...,...,...,...,...,...,...
20044,만랩커피,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 중대로23길 21,5호선 오금,124.273859,NaN,오금,오금역 만랩커피
20045,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,NaN,방화,방화역 얼음과자
20046,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,NaN,상일동,상일동역 오이소서울
20047,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,NaN,굽은다리,굽은다리역 얼음왕국명일14


In [40]:
items = data["검색어"][4242:]
items

4242             우장산역 왕세숫대야냉면
4243     둔촌동역 간단하지만특별한화피디Cafe
4244                아차산역 두부천국
4245      군자역 간단하지만특별한화피디Cafe
4246                 길동역 귀일교자
                 ...         
20044                오금역 만랩커피
20045                방화역 얼음과자
20046              상일동역 오이소서울
20047          굽은다리역 얼음왕국명일14
20048               상일동역 얼음과자
Name: 검색어, Length: 15807, dtype: object

In [41]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [42]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('link_photo > img') #이미지
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")


진행상황 : 1/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/15807
식당 존재 x
진행상황 : 3/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/15807
식당 존재 x
진행상황 : 5/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/15807
식당 존재 x
진행상황 : 8/15807
식당 존재 x
진행상황 : 9/15807
식당 존재 x
진행상황 : 10/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/15807
식당 존재 x
진행상황 : 13/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/15807
식당 존재 x
진행상황 : 16/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 17/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 18/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 19/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 20/15807
식당 존재 x
진행상황 : 21/15807
식당 존재 x
진행상황 : 22/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 23/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 24/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 25/15807


식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 195/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 196/15807
식당 존재 x
진행상황 : 197/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 198/15807
식당 존재 x
진행상황 : 199/15807
식당 존재 x
진행상황 : 200/15807
식당 존재 x
진행상황 : 201/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 202/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 203/15807
식당 존재 x
진행상황 : 204/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 205/15807
식당 존재 x
진행상황 : 206/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 207/15807
식당 존재 x
진행상황 : 208/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 209/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 210/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 211/15807
식당 존재 x
진행상황 : 212/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 213/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 214/15807
식당 존재 x
진행상황 : 215/15807
식당 존재 x
진행상황 : 216/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 217/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 

식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 384/15807
식당 존재 x
진행상황 : 385/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 386/15807
식당 존재 x
진행상황 : 387/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 388/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 390/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 392/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 393/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 394/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 395/15807
식당 존재 x
진행상황 : 396/15807
식당 존재 x
진행상황 : 397/15807
식당 존재 x
진행상황 : 398/15807
식당 존재 x
진행상황 : 399/15807
식당 존재 x
진행상황 : 400/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 401/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 402/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 403/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 404/15807
식당 존재 x
진행상황 : 405/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가

식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 573/15807
식당 존재 x
진행상황 : 574/15807
식당 존재 x
진행상황 : 575/15807
식당 존재 x
진행상황 : 576/15807
식당 존재 x
진행상황 : 577/15807
식당 존재 x
진행상황 : 578/15807
식당 존재 x
진행상황 : 579/15807
식당 존재 x
진행상황 : 580/15807
식당 존재 x
진행상황 : 581/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 582/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 583/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 584/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 585/15807
식당 존재 x
진행상황 : 586/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 587/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 588/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 589/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 590/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 591/15807
식당 존재 x
진행상황 : 592/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 593/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 594/15807
식당 존재 x
진행상황 : 595/15807
식당 존재 x
진행상황 : 596/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 59

식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 764/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 765/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 766/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 767/15807
식당 존재 x
진행상황 : 768/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 769/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 770/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 771/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 772/15807
식당 존재 x
진행상황 : 773/15807
식당 존재 x
진행상황 : 774/15807
식당 존재 x
진행상황 : 775/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 776/15807
식당 존재 x
진행상황 : 777/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 778/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 779/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 780/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 781/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 782/15807
식당 존재 x
진행상황 : 783/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 784/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 785/1580

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 951/15807
식당 존재 x
진행상황 : 952/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 953/15807
식당 존재 x
진행상황 : 954/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 955/15807
식당 존재 x
진행상황 : 956/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 957/15807
식당 존재 x
진행상황 : 958/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 959/15807
식당 존재 x
진행상황 : 960/15807
식당 존재 x
진행상황 : 961/15807
식당 존재 x
진행상황 : 962/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 963/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 964/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 965/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 966/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 967/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 968/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 969/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 970/15807
식당 존재 x
진행상황 : 971/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 972/15807
식당 존재 x
진행상황 : 973/1580

식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1132/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1133/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1134/15807
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1135/15807
식당 존재 x
진행상황 : 1136/15807
식당 존재 x
진행상황 : 1137/15807
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1138/15807
식당 존재 x
진행상황 : 1139/15807
식당 존재 x
진행상황 : 1140/15807
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1141/15807
식당 존재 x
진행상황 : 1142/15807
식당 존재 x
진행상황 : 1143/15807
식당 존재 x
진행상황 : 1144/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1145/15807
식당 존재 x
진행상황 : 1146/15807
식당 존재 x
진행상황 : 1147/15807
식당 존재 x
진행상황 : 1148/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1149/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1150/15807
식당 존재 x
진행상황 : 1151/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1152/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1153/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1154/15807
식당 존재 x
진행상황 : 1155/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1317/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1318/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1319/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1320/15807
식당 존재 x
진행상황 : 1321/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1322/15807
식당 존재 x
진행상황 : 1323/15807
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1324/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1325/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1326/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1327/15807
식당 존재 x
진행상황 : 1328/15807
식당 존재 x
진행상황 : 1329/15807
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1330/15807
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1331/15807


KeyboardInterrupt: 

In [43]:
img_elem

,ItemID,img_url
0,우장산역 왕세숫대야냉면,None
1,우장산역 왕세숫대야냉면,None
2,우장산역 왕세숫대야냉면,None
3,우장산역 왕세숫대야냉면,None
4,우장산역 왕세숫대야냉면,None
...,...,...
5447,굽은다리역 갸또마마,None
5448,굽은다리역 갸또마마,None
5449,굽은다리역 갸또마마,None
5450,굽은다리역 갸또마마,None


In [44]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,bee73c,우장산역 왕세숫대야냉면,3점,2021.04.04.
1,sa540j,우장산역 왕세숫대야냉면,5점,2021.04.02.
2,ndhfg3,우장산역 왕세숫대야냉면,1점,2021.04.02.
3,a3pqr8,우장산역 왕세숫대야냉면,5점,2021.02.22.
4,7kc4tr,우장산역 왕세숫대야냉면,4점,2021.02.22.
...,...,...,...,...
5447,1enlhhg,굽은다리역 갸또마마,3점,2020.12.20.
5448,19v4as4,굽은다리역 갸또마마,5점,2020.07.17.
5449,g76mqt,굽은다리역 갸또마마,5점,2020.07.17.
5450,e4kau9,굽은다리역 갸또마마,4점,2020.03.19.


In [45]:
img_elem

,ItemID,img_url
0,우장산역 왕세숫대야냉면,None
1,우장산역 왕세숫대야냉면,None
2,우장산역 왕세숫대야냉면,None
3,우장산역 왕세숫대야냉면,None
4,우장산역 왕세숫대야냉면,None
...,...,...
5447,굽은다리역 갸또마마,None
5448,굽은다리역 갸또마마,None
5449,굽은다리역 갸또마마,None
5450,굽은다리역 갸또마마,None


In [46]:
review_elem

,ItemID,review
0,우장산역 왕세숫대야냉면,가성비 나쁘지않음 맛도 그냥 쏘쏘
1,우장산역 왕세숫대야냉면,4천 5백원 돈까스 양 실화임? 후덜덜 하넹 고기도 두툼하다
2,우장산역 왕세숫대야냉면,다들 초딩 입맛임 ?
3,우장산역 왕세숫대야냉면,가성비 킹왈짱~~~ 곤까스 4500인데 맛나요
4,우장산역 왕세숫대야냉면,솔직히.. 가격싼맛에 한끼 때우기 좋은집입니다 맛은 뭐 개인적 취향이겠지만 가격...
...,...,...
5447,굽은다리역 갸또마마,20.12.20 기준 올리브빵은 비추입니다. 사과빵 무난하고 크루아상 안먹어봤지만 ...
5448,굽은다리역 갸또마마,"갸또마마 빵 안드신 분들은 다이어트 하시는건가요?? 진짜 맛있습니다,, 맛있어요 그..."
5449,굽은다리역 갸또마마,저 여기 일주일에 한번은 꼭 가는듯요 명일역 근방 최고 빵 맛집임니다 믿고 가세용
5450,굽은다리역 갸또마마,올리브빵 맛있네요. 평소 올리브 싫어해서 샌드위치 먹을 땐 다 빼고 먹고 피자에도 ...


In [47]:
img_elem.head()

,ItemID,img_url
0,우장산역 왕세숫대야냉면,None
1,우장산역 왕세숫대야냉면,None
2,우장산역 왕세숫대야냉면,None
3,우장산역 왕세숫대야냉면,None
4,우장산역 왕세숫대야냉면,None


In [48]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID      ItemID Rating    Timestamp
5447  1enlhhg  굽은다리역 갸또마마     3점  2020.12.20.
5448  19v4as4  굽은다리역 갸또마마     5점  2020.07.17.
5449   g76mqt  굽은다리역 갸또마마     5점  2020.07.17.
5450   e4kau9  굽은다리역 갸또마마     4점  2020.03.19.
5451   e4rpa1  굽은다리역 갸또마마     5점  2020.02.04.
          ItemID img_url
5447  굽은다리역 갸또마마    None
5448  굽은다리역 갸또마마    None
5449  굽은다리역 갸또마마    None
5450  굽은다리역 갸또마마    None
5451  굽은다리역 갸또마마    None
          ItemID                                             review
5447  굽은다리역 갸또마마  20.12.20 기준 올리브빵은 비추입니다. 사과빵 무난하고 크루아상 안먹어봤지만 ...
5448  굽은다리역 갸또마마  갸또마마 빵 안드신 분들은 다이어트 하시는건가요?? 진짜 맛있습니다,, 맛있어요 그...
5449  굽은다리역 갸또마마      저 여기 일주일에 한번은 꼭 가는듯요 명일역 근방 최고 빵 맛집임니다 믿고 가세용
5450  굽은다리역 갸또마마  올리브빵 맛있네요. 평소 올리브 싫어해서 샌드위치 먹을 땐 다 빼고 먹고 피자에도 ...
5451  굽은다리역 갸또마마                                      올리브치아바타 너무맛잇떵


In [49]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_rating(5575).csv')

In [50]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_User_ID(5575).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_img(5575).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_review(5575).csv')

In [89]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


1013

In [100]:
print (data.loc[data['검색어'] == "서대문역 아웃백스테이크하우스"])

             상호명 상권업종중분류명 상권업종소분류명             도로명주소       역명          거리  \
2527  아웃백스테이크하우스       양식  패밀리레스토랑  서울특별시 중구 통일로 114  5호선 서대문  122.856647   

      카카오url    역              검색어  
2527     NaN  서대문  서대문역 아웃백스테이크하우스  


IndexError: list index out of range